Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [3]:
sales = pd.read_csv(r'C:\Users\satyapattnaik\PythonCodes\KaggleCourse\competitive-data-science\competitive-data-science-predict-future-sales\sales_train.csv')

In [4]:
sales.head()

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [16]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
#print(grid)
#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False)['item_cnt_day'].agg(['sum']).rename(columns={'sum': 'target'})
#print(gb.columns)
#print([col[0] if col[-1]=='' else col[-1] for col in gb.columns.values])
#fix column names
#gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
print(gb.columns)
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

Index(['target'], dtype='object')


In [17]:
all_data.head()

shop_id  item_id  date_block_num  target
139255        0       19               0     0.0
141495        0       27               0     0.0
144968        0       28               0     0.0
142661        0       29               0     0.0
138947        0       32               0     6.0

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [18]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988963742


#### Method 2

In [19]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988963742


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [20]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [22]:
from sklearn import model_selection

In [59]:
all_data.drop(['item_target_enc'],axis=1,inplace=True)

In [60]:
all_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'target'], dtype='object')

In [62]:
train_new = pd.DataFrame(index=all_data.index,\
                        columns=['shop_id', 'item_id', 'date_block_num', 'target',\
                                'shop_id_mean_encoded', 'item_id_mean_encoded', 'date_block_num_mean_encoded'])

y_tr = all_data["target"]
kf = model_selection.KFold(5,shuffle=False)
for tr_ind,val_ind in kf.split(y_tr):
    X_train,X_val = all_data.iloc[tr_ind], all_data.iloc[val_ind]
    for each_col in ['shop_id', 'item_id', 'date_block_num']:
        means = X_val[each_col].map(X_train.groupby(each_col)["target"].mean())
        X_val.loc[:,f"{each_col}_mean_encoded"] = means
    train_new.iloc[val_ind] = X_val
prior = all_data["target"].mean()
train_new.fillna(prior,inplace=True)

In [64]:
train_new.head()

shop_id  item_id  date_block_num  target  shop_id_mean_encoded  \
139255      0.0     19.0             0.0     0.0              0.334273   
141495      0.0     27.0             0.0     0.0              0.334273   
144968      0.0     28.0             0.0     0.0              0.334273   
142661      0.0     29.0             0.0     0.0              0.334273   
138947      0.0     32.0             0.0     6.0              0.334273   

        item_id_mean_encoded  date_block_num_mean_encoded  
139255              0.334273                     0.334273  
141495              0.048523                     0.334273  
144968              0.142424                     0.334273  
142661              0.030303                     0.334273  
138947              0.894020                     0.334273

In [65]:
# YOUR CODE GOES HERE
encoded_feature = train_new["item_id_mean_encoded"].values
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.4164590488853805
Current answer for task KFold_scheme is: 0.4164590488853805


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [71]:
loo_sum = all_data.groupby('item_id')["target"].transform("sum")
loo_count = all_data.groupby('item_id')["target"].transform("count")
all_data["item_id_mean_encoded"] = (loo_sum-all_data["target"])/(loo_count)
all_data['item_id_mean_encoded'].fillna(0.3343, inplace=True) 
loo_count

139255        45
141495       739
144968       595
142661       321
138947      1586
            ... 
10768834     449
10769024     129
10769690     399
10771216    1176
10770511    1221
Name: target, Length: 10913850, dtype: int64

In [68]:
all_data.shape

(10913850, 4)

In [72]:
# YOUR CODE GOES HERE
encoded_feature = all_data["item_id_mean_encoded"].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.4803928066530125
Current answer for task Leave-one-out_scheme is: 0.4803928066530125


In [73]:
del loo_sum, loo_count

In [74]:
import gc
gc.collect()

153

# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [75]:
all_data.drop(["item_id_mean_encoded"],axis=1,inplace=True)
all_data.head()

shop_id  item_id  date_block_num  target
139255        0       19               0     0.0
141495        0       27               0     0.0
144968        0       28               0     0.0
142661        0       29               0     0.0
138947        0       32               0     6.0

In [76]:
alpha=100

smooth_mean = all_data.groupby('item_id')["target"].transform("mean")
smooth_count = all_data.groupby('item_id')["target"].transform("count")

all_data["item_id_mean_encoded"] = (smooth_mean * smooth_count + 0.3343*alpha) / (smooth_count + alpha)
all_data['item_id_mean_encoded'].fillna(0.3343, inplace=True)
all_data.head()

shop_id  item_id  date_block_num  target  item_id_mean_encoded
139255        0       19               0     0.0              0.237448
141495        0       27               0     0.0              0.089905
144968        0       28               0     0.0              0.168964
142661        0       29               0     0.0              0.107910
138947        0       32               0     6.0              1.260635

In [77]:
# YOUR CODE GOES HERE
encoded_feature = all_data["item_id_mean_encoded"].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.4818198797438016
Current answer for task Smoothing_scheme is: 0.4818198797438016


In [78]:
del smooth_mean, smooth_count
gc.collect()

68

# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

In [85]:
all_data.drop(["item_id_mean_encoded"],axis=1,inplace=True)
all_data.head()

shop_id  item_id  date_block_num  target
139255        0       19               0     0.0
141495        0       27               0     0.0
144968        0       28               0     0.0
142661        0       29               0     0.0
138947        0       32               0     6.0

In [86]:
cumsum = all_data.groupby('item_id')["target"].cumsum() - all_data['target']
cumcount = all_data.groupby('item_id')["target"].cumcount()

all_data["item_id_mean_encoded"] = cumsum/cumcount
all_data['item_id_mean_encoded'].fillna(0.3343, inplace=True)

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [87]:
# YOUR CODE GOES HERE
encoded_feature = all_data["item_id_mean_encoded"].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.5025245211433321
Current answer for task Expanding_mean_scheme is: 0.5025245211433321


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [88]:
STUDENT_EMAIL = "satyapattnaik76@gmail.com"
STUDENT_TOKEN = "2HHUbYyNq97l1LMn"
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.4164590488853805
Task Leave-one-out_scheme: 0.4803928066530125
Task Smoothing_scheme: 0.4818198797438016
Task Expanding_mean_scheme: 0.5025245211433321


In [89]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
